![Rearc](../rearc_logo_rgb.png)

 ## Demo: EDGAR Company Filing Dataset - Form Type 10-K | SEC 

This dataset contains the quarterly reports of all company filings of form "10-K" since 1993. [Check it out](https://aws.amazon.com/marketplace/pp/prodview-k65p3l3pdyqfw?qid=1622601179293&sr=0-1&ref_=srh_res_product_title) on Amazon Data Exchange marktplace.

Form 10-K is a ...

To download this file-based data product through the AWS Data Exchange Marketplace, you must subscribe to the product. For details, refer to [Subscribing to Data Products](https://docs.aws.amazon.com/data-exchange/latest/userguide/subscribe-to-data-sets.html) on AWS Data Exchange in the AWS Data Exchange User Guide.

After subscribing to the product, you can use this notebook to export the dataset revisions and explore the data. This dataset contains the 10-K forms submitted to SEC as provided by the EDGAR portal. All files are in .txt format.

### Export Dataset Revisions to S3

You can use the following code snippet to export any number of desired revisions from this dataset. There are two input avariables required for this: 
1. The name of the <b>S3 bucket</b> that you would like the data to be exported to.
2. The <b>dataset ARN</b> for this product which you can access from the AWS Data Exchange console after you subscribe to the produc.

In [152]:
import boto3
import os
import re 
import time
import click
import uuid

dx = boto3.client('dataexchange', region_name='us-east-1')
s3 = boto3.client('s3')


def get_all_revisions(data_set_id):

    revisions = []
    res = dx.list_data_set_revisions(DataSetId=data_set_id)
    next_token = res.get('NextToken')
    
    revisions += res.get('Revisions')
    while next_token:
        res = dx.list_data_set_revisions(DataSetId=data_set_id,
                                         NextToken=next_token)
        revisions += res.get('Revisions')
        next_token = res.get('NextToken')
        
    return revisions


def get_all_assets(data_set_id, revision_id):
    assets = []
    res = dx.list_revision_assets(DataSetId=data_set_id,
                                  RevisionId=revision_id)
    next_token = res.get('NextToken')
    
    assets += res.get('Assets')
    while next_token:
        res = dx.list_revision_assets(DataSetId=data_set_id,
                                      RevisionId=revision_id,
                                      NextToken=next_token)
        assets += res.get('Assets')
        next_token = res.get('NextToken')
        
    return assets


def get_entitled_data_sets():
    data_sets = []
    res = dx.list_data_sets(Origin='ENTITLED')
    next_token = res.get('NextToken')
    
    data_sets += res.get('DataSets')
    while next_token:
        res = dx.list_data_sets(Origin='ENTITLED',
                                NextToken=next_token)
        data_sets += res.get('DataSets')
        next_token = res.get('NextToken')
        
    return data_sets


def export_assets(assets, bucket):
    
    asset_destinations = []

    for asset in assets:
        asset_destinations.append({
            "AssetId": asset.get('Id'),
            "Bucket": bucket,
            "Key": asset.get('Name')
        })

    job = dx.create_job(Type='EXPORT_ASSETS_TO_S3', Details={
        "ExportAssetsToS3": {
            "RevisionId": asset.get("RevisionId"), "DataSetId": asset.get("DataSetId"),
            "AssetDestinations": asset_destinations
        }
    })

    job_id = job.get('Id')
    dx.start_job(JobId=job_id)

    while True:
        job = dx.get_job(JobId=job_id)

        if job.get('State') == 'COMPLETED':
            break
        elif job.get('State') == 'ERROR':
            raise Exception("Job {} failed to complete - {}".format(
                job_id, job.get('Errors')[0].get('Message'))
            )

        time.sleep(1)


def to_url(s):
    s = re.sub(r"[^\w\s]", '', s)
    s = re.sub(r"\s+", '-', s)

    return s


def download_assets(assets, bucket, asset_dir):
    for asset in assets:
        asset_name = asset.get('Name')
        sub_dir = os.path.dirname(asset_name)
        full_dir = os.path.join(asset_dir, sub_dir)

        if not os.path.exists(full_dir):
            os.makedirs(full_dir)

        asset_file = os.path.join(full_dir, os.path.basename(asset_name))

        s3.download_file(bucket, asset_name, asset_file)

        print("Downloaded file {}".format(asset_file))


def make_s3_staging_bucket():
    bucket_name = str(uuid.uuid4())
    s3.create_bucket(Bucket=bucket_name)
    return bucket_name


def remove_s3_bucket(bucket_name):
    s3_resource = boto3.resource('s3')
    bucket = s3_resource.Bucket(bucket_name)
    bucket.objects.all().delete()
    bucket.delete()


@click.command()
@click.option('--s3-bucket', '-s')
def main(s3_bucket):

    if not s3_bucket:
        print("No s3 bucket provided, creating temporary staging bucket")
        temp_bucket = make_s3_staging_bucket()
        print("Created temporary bucket {}".format(temp_bucket))

    try:
        data_sets = get_entitled_data_sets()

        staging_bucket = s3_bucket or temp_bucket

        for ds in data_sets:
            print("Getting all Assets for Data set ### {} ###".format(ds.get('Name')))

            revisions = get_all_revisions(ds.get('Id'))
            for rev in revisions:
                assets = get_all_assets(ds.get('Id'), rev.get('Id'))

                destination_dir = os.path.join(to_url(ds.get('Name')), rev.get('Id'))

                export_assets(assets, staging_bucket)
                download_assets(assets, staging_bucket, destination_dir)

            print("---")
    finally:
        if temp_bucket:
            print("Removing temporary bucket {}".format(temp_bucket))
            remove_s3_bucket(temp_bucket)

In [6]:
import os
import sys
import time
import json
import boto3
from datetime import date, datetime
from multiprocessing.dummy import Pool
from itertools import product

s3_bucket = 'adx-subscribed-datasets'
dataset_arn = 'arn:aws:dataexchange:us-east-1::data-sets/65037f8cc04e824742b9ab1afa73cd7d'

NUM_CONCURRENT_JOBS = 10

def export_and_download_assets(al, s3_bucket, dataset_name, revision_id):
    export_assets(al, s3_bucket)

data_sets = get_entitled_data_sets()

dataset = None
dataset_name = ''
dataset_id = ''

for ds in data_sets:
    if ds['Arn'] == dataset_arn:
        dataset = ds
        dataset_name = dataset.get('Name')
        dataset_id = dataset.get('Id')
        
        print(f'Dataset Name: {dataset_name}')

try:
    print("Getting all Assets for Data set ### {} ###".format(dataset_name))

    revisions_all = get_all_revisions(dataset_id)
    revisions = revisions_all #[0:4]
    print(f'Num Revisions: {len(revisions)}')
          
    for i, rev in enumerate(revisions):
        revision_id = rev.get('Id')
        asset_list = get_all_assets(dataset_id, revision_id)
        num_assets = len(asset_list)
        print(num_assets)

        if type(asset_list) == list:
            if num_assets == 0:
                print(f'No assets in revision {revision_id}')
                sys.exit()

        # chunck into 10k bundles to account for ADX's limit of 10k assets per revisions
        asset_lists_10k = [asset_list[i:i+10000] for i in range(0, num_assets, 10000)]

        i = 0
        for assets_10k in asset_lists_10k:

            # chunck into 100 bundles to account for ADX's limit of 100 assets per job
            asset_lists_100 = [assets_10k[i:i+100] for i in range(0, len(assets_10k), 100)]

            asset_lists_100_formatted = []
            
            
            with (Pool(NUM_CONCURRENT_JOBS)) as p:
                print(f'len of asset_list: {len(asset_lists_100)}')
                map_input = product(asset_lists_100, [s3_bucket], [dataset_name], [revision_id])
                map_input_list = list(product(asset_lists_100, [s3_bucket], [dataset_name], [revision_id]))
                print(f'len of product: {len(map_input_list)}')
                p.starmap(export_and_download_assets, map_input)

except Exception as e:
    print(e)

Getting all Assets for Data set ### EDGAR Company Filing Dataset - Form Type 10-K | SEC ###
Num Revisions: 110
7
len of asset_list: 1
len of product: 1
449
len of asset_list: 5
len of product: 5
568
len of asset_list: 6
len of product: 6
1312
len of asset_list: 14
len of product: 14
5282
len of asset_list: 53
len of product: 53
741
len of asset_list: 8
len of product: 8
596
len of asset_list: 6
len of product: 6
1556
len of asset_list: 16
len of product: 16
5025
len of asset_list: 51
len of product: 51
593
len of asset_list: 6
len of product: 6
614
len of asset_list: 7
len of product: 7
1771
len of asset_list: 18
len of product: 18
5121
len of asset_list: 52
len of product: 52
641
len of asset_list: 7
len of product: 7
649
len of asset_list: 7
len of product: 7
1366
len of asset_list: 14
len of product: 14
5527
len of asset_list: 56
len of product: 56
701
len of asset_list: 8
len of product: 8
793
len of asset_list: 8
len of product: 8
1539
len of asset_list: 16
len of product: 16
5735

## Dataset Directory Structure

After you exported the revisions, if you navigate to the S3 bucket you specified above to export the data, you will find the dataset structured as following:
`
dataset
     > 2020
        > Q1
              > file1.txt
              > file2.txt
              > ...
        > Q2
        > Q3
        > Q4
    > 2019
        > Q1
        > Q2
        > Q3
        > Q4
    > ...
        > ...
    > 1993
        > ...
`

## Extract financial tables for sections 6, 7 from a 10-K form

In [153]:
import os
import re
from bs4 import BeautifulSoup
import pandas as pd

In [154]:
data_dir = os.path.join(os.getcwd(), 'EDGAR-Company-Filing-Dataset-Form-Type-10K-SEC/c26c517bea98245dc7a949484f4cac5a/edgar/edgar_company_filings_form_10_K/dataset/2020/QTR4')

# demo/EDGAR-Company-Filing-Dataset-Form-Type-10K-SEC/c26c517bea98245dc7a949484f4cac5a/edgar/edgar_company_filings_form_10_K/dataset/2020/QTR4/1385157-0001558370-20-013511.txt

data = None
for root, dirs, files in os.walk(data_dir):
    for f in files:
        if f.endswith('.txt') and str(f) == '1385157-0001558370-20-013511.txt':
            with open(os.path.join(data_dir, f)) as reader:
                data = reader.read()
            break
    break
    
raw_10k = data

In [161]:
print(raw_10k[0:1066])

<SEC-DOCUMENT>0001558370-20-013511.txt : 20201110
<SEC-HEADER>0001558370-20-013511.hdr.sgml : 20201110
<ACCEPTANCE-DATETIME>20201110152800
ACCESSION NUMBER:		0001558370-20-013511
CONFORMED SUBMISSION TYPE:	10-K
PUBLIC DOCUMENT COUNT:		133
CONFORMED PERIOD OF REPORT:	20200925
FILED AS OF DATE:		20201110
DATE AS OF CHANGE:		20201110

FILER:

	COMPANY DATA:	
		COMPANY CONFORMED NAME:			TE Connectivity Ltd.
		CENTRAL INDEX KEY:			0001385157
		STANDARD INDUSTRIAL CLASSIFICATION:	WHOLESALE-ELECTRONIC PARTS & EQUIPMENT, NEC [5065]
		IRS NUMBER:				980518048
		STATE OF INCORPORATION:			V8
		FISCAL YEAR END:			0925

	FILING VALUES:
		FORM TYPE:		10-K
		SEC ACT:		1934 Act
		SEC FILE NUMBER:	001-33260
		FILM NUMBER:		201300918

	BUSINESS ADDRESS:	
		STREET 1:		MUHLENSTRASSE 26
		CITY:			SCHAFFHAUSEN
		STATE:			V8
		ZIP:			CH-8200
		BUSINESS PHONE:		41 (0)52 633 6661

	MAIL ADDRESS:	
		STREET 1:		MUHLENSTRASSE 26
		CITY:			SCHAFFHAUSEN
		STATE:			V8
		ZIP:			CH-8200

	FORMER COMPANY:	
		FORMER CON

In [162]:
# Regex to find <DOCUMENT> tags
doc_start_pattern = re.compile(r'<DOCUMENT>')
doc_end_pattern = re.compile(r'</DOCUMENT>')

# Regex to find <TYPE> tag prceeding any characters, terminating at new line
type_pattern = re.compile(r'<TYPE>[^\n]+')

In [163]:
document = {}

# Create a loop to go through each section type and save only the 10-K section in the dictionary
for doc_type, doc_start, doc_end in zip(doc_types, doc_start_is, doc_end_is):
    if doc_type == '10-K':
        document[doc_type] = raw_10k[doc_start:doc_end]

In [164]:
document['10-K'][0:500]

'\n<?xml version=\'1.0\' encoding=\'UTF-8\'?>\n\n      <!-- iXBRL document created with: Toppan Merrill Bridge iXBRL 9.6.7556.35618 -->\n      <!-- Based on: iXBRL 1.1 -->\n      <!-- Created on: 11/10/2020 8:25:25 PM -->\n      <!-- iXBRL Library version: 1.0.7556.35625 -->\n      <!-- iXBRL Service Job ID: 304cf6e6-6495-496f-929f-ee1dbaf3e5c1 -->\n\n  <html xmlns:us-gaap="http://fasb.org/us-gaap/2020-01-31" xmlns:link="http://www.xbrl.org/2003/linkbase" xmlns:tel="http://www.te.com/20200925" xmlns:country="'

## Apply REGEX to find Sections 6 and 7 under 10-K Section 

The items in this `document` can be found in four different patterns. The specified regex patterns accounts for various formats that a section title may appear in the document.

In [166]:
regex = re.compile(r'(>Item(\s|&#160;|&nbsp;)([1-9])([0-9]){0,1}[AB]{0,1}\.{0,1})|(ITEM\s{0,1}(&#160;|&nbsp;){0,1}([1-9])([0-9]){0,1}[AB]{0,1}\.{0,1})')

matches = regex.finditer(document['10-K'])

for match in matches:
    print(match)

<re.Match object; span=(364584, 364590), match='ITEM1B'>
<re.Match object; span=(364751, 364764), match='>Item&#160;1.'>
<re.Match object; span=(364975, 364981), match='ITEM1B'>
<re.Match object; span=(365589, 365595), match='ITEM1A'>
<re.Match object; span=(365760, 365774), match='>Item&#160;1A.'>
<re.Match object; span=(365985, 365991), match='ITEM1A'>
<re.Match object; span=(366772, 366781), match='>Item 1B.'>
<re.Match object; span=(367622, 367627), match='ITEM2'>
<re.Match object; span=(367791, 367804), match='>Item&#160;2.'>
<re.Match object; span=(368015, 368020), match='ITEM2'>
<re.Match object; span=(368634, 368639), match='ITEM3'>
<re.Match object; span=(368809, 368822), match='>Item&#160;3.'>
<re.Match object; span=(369033, 369038), match='ITEM3'>
<re.Match object; span=(369830, 369838), match='>Item 4.'>
<re.Match object; span=(372306, 372311), match='ITEM5'>
<re.Match object; span=(372493, 372506), match='>Item&#160;5.'>
<re.Match object; span=(372717, 372722), match='ITEM

In [167]:
# Matches
matches = regex.finditer(document['10-K'])

# Create the dataframe
test_df = pd.DataFrame([(x.group(), x.start(), x.end()) for x in matches])

test_df.columns = ['item', 'start', 'end']
test_df['item'] = test_df.item.str.lower()

# Display the dataframe
test_df.head()

,item,start,end
0,item1b,364584,364590
1,>item&#160;1.,364751,364764
2,item1b,364975,364981
3,item1a,365589,365595
4,>item&#160;1a.,365760,365774


In [168]:
# Get rid of unnesesary charcters from the dataframe
test_df.replace('&#160;',' ',regex=True,inplace=True)
test_df.replace('&nbsp;',' ',regex=True,inplace=True)
test_df.replace(' ','',regex=True,inplace=True)
test_df.replace('\.','',regex=True,inplace=True)
test_df.replace('>','',regex=True,inplace=True)

# display the dataframe
print(test_df.head(30))
print(test_df.shape)

      item   start     end
0   item1b  364584  364590
1    item1  364751  364764
2   item1b  364975  364981
3   item1a  365589  365595
4   item1a  365760  365774
5   item1a  365985  365991
6   item1b  366772  366781
7    item2  367622  367627
8    item2  367791  367804
9    item2  368015  368020
10   item3  368634  368639
11   item3  368809  368822
12   item3  369033  369038
13   item4  369830  369838
14   item5  372306  372311
15   item5  372493  372506
16   item5  372717  372722
17   item6  373458  373463
18   item6  373638  373651
19   item6  373862  373867
20   item7  374505  374510
21   item7  374692  374705
22   item7  374916  374921
23  item7a  375634  375640
24  item7a  375821  375835
25  item7a  376046  376052
26   item8  376731  376736
27   item8  376918  376931
28   item8  377142  377147
29   item9  377976  377984
(74, 3)


In [170]:
# Drop duplicates and only keep the last appearance of each section
pos_dat = test_df.sort_values('start', ascending=True).drop_duplicates(subset=['item'], keep='last')

# Set item as the dataframe index
pos_dat.set_index('item', inplace=True)

pos_dat

,start,end
item,,
item1,364751,364764
item1a,492441,492447
item1b,591188,591196
item2,592784,592789
item3,615049,615054
item4,617402,617409
item5,619342,619347
item6,692256,692261
item7,820366,820371


In [173]:
# Get desired sections
item_6_raw = document['10-K'][pos_dat['start'].loc['item6']:pos_dat['start'].loc['item7']]   
item_7_raw = document['10-K'][pos_dat['start'].loc['item7']:pos_dat['start'].loc['item7a']]   

# Convert the raw text we have to exrtacted to BeautifulSoup object 
item_6_content = BeautifulSoup(item_6_raw, 'lxml')
item_7_content = BeautifulSoup(item_7_raw, 'lxml')

In [175]:
print(item_7_content.prettify()[0:1000])

<html>
 <body>
  <p>
   ITEM7MANAGEMENTSDISCUSSIONANDANALYSISOFF"&gt;
  </p>
  <p style="font-family:'Times New Roman','Times','serif';font-size:10pt;line-height:1.19;margin:0pt 0pt 12pt 0pt;">
   <b style="font-weight:bold;">
    ITEM
   </b>
   <b style="font-weight:bold;white-space:pre-wrap;">
    7.  MANAGEMENT’S DISCUSSION AND ANALYSIS OF FINANCIAL CONDITION AND RESULTS OF OPERATIONS
   </b>
  </p>
  <p style="font-family:'Times New Roman','Times','serif';font-size:10pt;line-height:1.19;text-indent:36pt;margin:0pt 0pt 12pt 0pt;">
   The following discussion and analysis of our financial condition and results of operations should be read in conjunction with our Consolidated Financial Statements and the accompanying notes included elsewhere in this Annual Report. The following discussion may contain forward-looking statements that reflect our plans, estimates, and beliefs. Our actual results could differ materially from those discussed in these forward-looking statements. Factors th

In [241]:
tables_6 = item_6_content.find_all('table')
tables_7 = item_7_content.find_all('table')

## Financial Data Summary from Section 6

In [242]:
from IPython.core.display import display, HTML

for item in tables_6:
    display(HTML(item.prettify()))

(1),Fiscal 2016 was a 53-week year.


(2),Fiscal 2016 included a pre-tax gain of $144 million on the sale of our Circuit Protection Devices business.


(3),Fiscal 2020 included a goodwill impairment charge related to the Sensors reporting unit in our Transportation Solutions segment. See Note 8 to the Consolidated Financial Statements for additional information regarding the impairment of goodwill.


(4),Fiscal 2016 net other income (expense) was recorded primarily pursuant to the Tax Sharing Agreement with Tyco International plc and Covidien plc and included $604 million of other expense related to the effective settlement of tax matters for the years 1997 through 2000 and $46 million of other expense related to a tax settlement in another tax jurisdiction.


(5),"For fiscal 2020, 2019, and 2018, see Note 16 to the Consolidated Financial Statements for additional information. Fiscal 2016 included a $1,135 million income tax benefit related to the effective settlement of tax matters for the years 1997 through 2000, partially offset by a $91 million income tax charge related to an increase to the valuation allowance for certain U.S. deferred tax assets. Additionally, fiscal 2016 included an $83 million net income tax benefit related to tax settlements in certain other tax jurisdictions, partially offset by an income tax charge related to certain legal entity restructurings."


(6),"Fiscal 2019 included a pre-tax loss of $86 million on the sale of our Subsea Communications business. For additional information regarding discontinued operations, see Note 4 to the Consolidated Financial Statements."


## Financial Tables of Section 7

### Products and Services

In [244]:
products = tables_7[12]
display(HTML(products.prettify()))

### Changes in Net Sales in Fiscal Year 2020 vs Fiscal Year 2019

In [245]:
product_changes = tables_7[13]
display(HTML(product_changes.prettify()))

### 

In [246]:
tables = tables_7[19:22]
for table in tables:
    display(HTML(table.prettify()))

## Extracting Numerical Data From Financial Summary Table in Section 6 in a  DataFrame

In [283]:
digits = [str(x) for x in range(10)]

rows = []
titles = []
row_number = -1
prev = 0

for item in tables_6:
    ii = item.find_all('td')
    for iii in ii:
        txt = iii.get_text().replace("\n",'').replace("\t",'').replace('​','')
        if txt:
            ll = txt.strip().split('\n')
            for l in ll:
                l = l.strip()
                if l and l != '$':
                    if l[0] in digits or (l[0] == '(' and l[1] in digits) or l.startswith('—'):
                        rows[row_number].append(l)
                        prev = 0
                    else:
                        if not prev:
                            row_number += 1
                            rows.append([])
                            titles.append(l)
                        else:
                            titles[-1] = titles[-1] + ' ' + l
                        prev = 1
    break

In [290]:
for ri, row in enumerate(rows):
    for si, s in enumerate(row):
        ss = s.replace(',', '')
        if ss.startswith('('):
            ss = '-' + ss[1:-1]       
            rows[ri][si] = ss


In [291]:
table_rows = [ [titles[i]] + rows[i] for i in range(len(titles))]
tables_df = pd.DataFrame(table_rows).set_index(0)

tables_df

,1,2,3,4,5
0,,,,,
As of or for Fiscal,2020,2019,2018,2017,2016(1)
"(in millions, except per share data) Statement of Operations Data Net sales","12,172","13,448","13,988","12,185","11,352"
Acquisition and integration costs,36,27,14,6,22
"Restructuring and other charges (credits), net(2)",257,255,126,147,-2
Impairment of goodwill(3),900,—,—,—,—
"Other income (expense), net(4)",20,2,1,-42,-677
Income tax (expense) benefit(5),-783,15,344,-180,826
Income (loss) from continuing operations,-259,"1,946","2,584","1,540","1,847"
"Income (loss) from discontinued operations, net of income taxes(6)",18,-102,-19,143,162
